# 2025 DL Lab8: RL Assignment_Super Mario World

Before we start, please put **your name** and **SID** in following format: <br>
Hi I'm 陸仁賈, 314831000.

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement
should do something in the beginning (monster attack)
Custom PPO implement
pre train weight 差不多，主要是 reward function
model weight capacity 1GB
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [2]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [ ]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
TOTAL_STEPS = 200_000
TRAIN_CHUNK = 50_000
N_ENVS = 2
LEARNING_RATE = 1e-4

# Evaluation & Recording Settings
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 18000

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR = os.path.join(LOG_DIR, "videos")
CKPT_DIR = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [ ]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env


## Initialize Env & Model

In [ ]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

# 2. Initialize Model
model = CustomPPO(
    VisionBackbonePolicy,
    train_env,
    policy_kwargs=dict(normalize_images=False),
    n_epochs=10,
    n_steps=512,
    batch_size=512,
    learning_rate=LEARNING_RATE,
    verbose=1,
    gamma=0.99,
    kl_coef=1,
    clip_range=0.5,
    tensorboard_log=TENSORBOARD_LOG,
)

## Training Loop

In [ ]:
best_mean = -1e18
trained = 0
round_idx = 0

try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False)
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"CustomPPO_step_{trained}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Save Best Model ---
        if mean_ret > best_mean:
            best_mean = mean_ret
            best_path = os.path.join(LOG_DIR, "best_model.zip")
            model.save(best_path)
            print(f"New best record. Saved to {best_path}")

        # --- Record Video ---
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"step_{trained}_mean_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")

## Display Video

In [ ]:
from IPython.display import Video
import glob

list_of_files = glob.glob(os.path.join(VIDEO_DIR, '*.mp4')) 
if list_of_files:
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"Playing: {latest_file}")
    display(Video(latest_file, embed=True, width=600))
else:
    print("No videos found yet.")